In [93]:
from array import array
import numpy as np
import random
import struct
from sklearn.metrics import confusion_matrix
import os
import cv2
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import pickle

In [94]:
def read(fname_img, fname_lbl):
    f = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", f.read(8))
    lbl = array("b", f.read())
    f.close()

    f = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", f.read(16))
    img = array("B", f.read())
    f.close()

    tmp = []
    cur_img = []
    img_matrix = []
    for x in img.tolist():
        tmp.append(x)
        if len(tmp) == 28:
            cur_img.append(tmp)
            tmp = []
        if len(cur_img) == 28:
            img_matrix.append(cur_img)
            cur_img = []

    return lbl.tolist(), img_matrix

In [95]:
train_size = 60000

train_lbl, train_img = read("train-images-idx3-ubyte", "train-labels-idx1-ubyte")
test_lbl, test_img = read("t10k-images-idx3-ubyte", "t10k-labels-idx1-ubyte")

index_shuf = range(train_size)
random.shuffle(index_shuf)
x = []
y = []
for i in range(train_size):
    x.append(train_img[i])
    y.append(train_lbl[i])
train_lbl, train_img = y, x

train_img = np.uint8(np.array(train_img))
test_img = np.uint8(np.array(test_img))

In [96]:
def cal_sift(in_img):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(in_img, None)
    return kp, des

In [97]:
def sift_sample(imgs, sample_num):
    key_points = np.array([])
    descriptors = np.array([])
    index = []
    for i in range(sample_num):
        kp, des = cal_sift(np.array(imgs[i]))
        if (des == None):
            continue
        if len(key_points):
            key_points = np.append(key_points, kp)
            descriptors = np.append(descriptors, des, axis=0)
            index = index + [i] * len(kp)
        else:
            key_points = kp
            descriptors = des
            index = [i] * len(kp)
        if (i+1) % 1000 == 0:
            print i+1, "sampled"
    return descriptors

In [98]:
def descriptor_cluster(descriptors, cluster_num, iter_nume):
    cluster = KMeans(n_clusters=cluster_num, max_iter=iter_num).fit(descriptors)
    return cluster

In [99]:
cluster_num = 50
sample_num = 5000
iter_num = 300

descriptors = sift_sample(train_img, sample_num)
print descriptors.shape
cluster = descriptor_cluster(descriptors, cluster_num, iter_num)

1000 sampled
2000 sampled
3000 sampled
4000 sampled
5000 sampled
(39319, 128)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [100]:
def img_to_feature(img, cluster, cluster_num):
    sift = cv2.xfeatures2d.SIFT_create()
    feature_layer1 = [0.0] * cluster_num
    feature_layer2 = [0.0] * cluster_num * 4
    feature_layer3 = [0.0] * cluster_num * 16
    
    kp, des = cal_sift(img)
    if des != None:
        cluster_result = cluster.predict(des)
        for c in cluster_result:
            feature_layer1[c] += 0.5
            
    cur = 0
    for i in range(2):
        for j in range(2):
            cur += 1
            crop_img = img[i*14 : (i+1)*14 -1, j*14 : (j+1)*14 -1]
            kp, des = cal_sift(crop_img)
            if des != None:
                cluster_result = cluster.predict(des)
                for c in cluster_result:
                    feature_layer2[(cur-1)*cluster_num + c] += 0.25
                    
    cur = 0
    for i in range(4):
        for j in range(4):
            cur += 1
            crop_img = img[i*7 : (i+1)*7 -1, j*7 : (j+1)*7 -1]
            kp, des = cal_sift(crop_img)
            if des != None:
                cluster_result = cluster.predict(des)
                for c in cluster_result:
                    feature_layer3[(cur-1)*cluster_num + c] += 0.25
    
    feature = feature_layer1 + feature_layer2 + feature_layer3
    return np.array(feature)

In [101]:
def generate_features(imgs, cluster, cluster_num):
    features = []
    for img in imgs:
        f = img_to_feature(img, cluster, cluster_num)
        features.append(f)
    return np.array(features)

In [102]:
features_train = generate_features(train_img, cluster, cluster_num)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:30: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [103]:
features_test = generate_features(test_img, cluster, cluster_num)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:30: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [115]:
def train_and_test(features_train, features_test, lbl_train, lbl_test):
    row_sums = features_train.sum(axis=1, dtype=np.float64)
    for i in range(len(row_sums)):
        if (row_sums[i] == 0):
            row_sums[i] = 1
    features_train = features_train / row_sums[:, np.newaxis]
    
    row_sums = features_test.sum(axis=1, dtype=np.float64)
    for i in range(len(row_sums)):
        if (row_sums[i] == 0):
            row_sums[i] = 1
    features_test = features_test / row_sums[:, np.newaxis]
    
    clf = svm.LinearSVC()
    clf.fit(features_train, lbl_train)
    result = clf.predict(features_test)
    error = sum([int(result[i] != test_lbl[i]) for i in range(len(result))])
    print "Accuracy: ", 1 - float(error)/len(result)
    print confusion_matrix(result, test_lbl)
    return clf

In [116]:
train_and_test(features_train, features_test, train_lbl, test_lbl)

Accuracy:  0.7678
[[ 889    3   55   16    9   59   72   21   24   34]
 [   5 1106   22    4   19   14   22   64    4   13]
 [   5    5  712   25   35   16   18   47   23   10]
 [   6    1   51  779   15   78   32   23   35   17]
 [   2    6   35   23  742   33   46   39   30   58]
 [  16    1   23   71   14  586   58   18   21   26]
 [  24    1   21   23   21   54  579   15   32   36]
 [   8    6   79   41   20   16   30  777    7   28]
 [  16    0   23   14   26   13   27   11  759   38]
 [   9    6   11   14   81   23   74   13   39  749]]


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)